In [1]:
import pandas as pd
import os
import re

In [2]:
GH_families= ['GH1','GH2','GH5','GH10','GH26','GH30','GH39','GH51','GH148','GH6','GH3','GH44','GH13','GH124','GH116','GH48','GH8','GH9','GH45','GH16','GH7','GH12','GH131']


In [3]:
#getting all the file names
import glob
hmm_files=glob.glob('./HMMs/*.hmm')
fasta_files=glob.glob('all*.fasta')

In [4]:
hmm_files.sort()
hmm_files

['./HMMs/GH10_80_no_outliers.hmm',
 './HMMs/GH116_80_no_outliers.hmm',
 './HMMs/GH12_80_no_outliers.hmm',
 './HMMs/GH131_80_no_outliers.hmm',
 './HMMs/GH13_80_no_outliers.hmm',
 './HMMs/GH148_80_no_outliers.hmm',
 './HMMs/GH16_80_no_outliers.hmm',
 './HMMs/GH1_80_no_outliers.hmm',
 './HMMs/GH26_80_no_outliers.hmm',
 './HMMs/GH2_80_no_outliers.hmm',
 './HMMs/GH30_80_no_outliers.hmm',
 './HMMs/GH39_80_no_outliers.hmm',
 './HMMs/GH3_80_no_outliers.hmm',
 './HMMs/GH44_80_no_outliers.hmm',
 './HMMs/GH45_80_no_outliers.hmm',
 './HMMs/GH48_80_no_outliers.hmm',
 './HMMs/GH51_80_no_outliers.hmm',
 './HMMs/GH5_80_no_outliers.hmm',
 './HMMs/GH6_60_no_outliers.hmm',
 './HMMs/GH7_60_no_outliers.hmm',
 './HMMs/GH8_80_no_outliers.hmm',
 './HMMs/GH9_80_no_outliers.hmm']

In [5]:
#setting threshold values
threshold_df=pd.read_csv('thresholds.csv')
thresholds=threshold_df['Bit-score threshold'].to_list()
threshold_dic= dict(zip(hmm_files, thresholds))
threshold_dic

{'./HMMs/GH10_80_no_outliers.hmm': 200,
 './HMMs/GH116_80_no_outliers.hmm': 34,
 './HMMs/GH12_80_no_outliers.hmm': 120,
 './HMMs/GH131_80_no_outliers.hmm': 387,
 './HMMs/GH13_80_no_outliers.hmm': 91,
 './HMMs/GH148_80_no_outliers.hmm': 44,
 './HMMs/GH16_80_no_outliers.hmm': 88,
 './HMMs/GH1_80_no_outliers.hmm': 352,
 './HMMs/GH26_80_no_outliers.hmm': 90,
 './HMMs/GH2_80_no_outliers.hmm': 215,
 './HMMs/GH30_80_no_outliers.hmm': 269,
 './HMMs/GH39_80_no_outliers.hmm': 226,
 './HMMs/GH3_80_no_outliers.hmm': 251,
 './HMMs/GH44_80_no_outliers.hmm': 339,
 './HMMs/GH45_80_no_outliers.hmm': 108,
 './HMMs/GH48_80_no_outliers.hmm': 869,
 './HMMs/GH51_80_no_outliers.hmm': 342,
 './HMMs/GH5_80_no_outliers.hmm': 55,
 './HMMs/GH6_60_no_outliers.hmm': 363,
 './HMMs/GH7_60_no_outliers.hmm': 436,
 './HMMs/GH8_80_no_outliers.hmm': 241,
 './HMMs/GH9_80_no_outliers.hmm': 391}

In [6]:
#all_sequences vs all hmms
ids=[]
for i,hmm in enumerate(hmm_files): 
    for j,fasta_file in enumerate(fasta_files):
        eval_file=hmm.split('/')[2].split('.')[0]+'_vs_'+fasta_file.split('.')[0]+'.txt'
        threshold=str(threshold_dic[hmm])
        cmd_search='hmmsearch --noali --domT ' +threshold +' --domtblout ' + eval_file+' '+ hmm+' '+fasta_file
        os.system(cmd_search)
        
        use, no_use=[],[]
        with open(eval_file,'r',encoding='utf-8') as otp:
            for ii in otp:
                if ii.startswith('#'):
                    #print(no_use)
                    no_use.append(ii)
                else:
                    #print(use)
                    use.append(ii) 
        xx=len(use)
        if xx>0:
            new_data=[]
            for i in use:
                j=re.split('\s+',i.strip())
                new_data.append(j)
            df2=pd.DataFrame(new_data)
            df3=df2[[0,13,15,16,17,18,22]]
            df3.columns=['ID','Score','hmm_start','hmm_end','ali_start','ali_end','description']
            x=df3['description'].tolist()
            y=df3['ID'].tolist()
            filename4=eval_file.split('.')[0]+'_raw.csv'
            df3.to_csv(filename4)
            IDs=[]
            for i in range(len(x)):
                if x[i].startswith('-'):
                    z=y[i]
                    IDs.append(z)
                else:
                    z=y[i]+x[i]
                    IDs.append(z)
            df3['IDs']=IDs
            df3=df3.drop(['ID','description'], axis=1)
            df3.Score = df3.Score.astype(float)
            df3=df3.sort_values(by=['Score'], ascending=False)
            
            #score=df3['Score'].to_list()
            data=df3['IDs'].to_list()
            #num_of_hits=len(set(data))
            #max_score=max(score)
            #min_score=min(score)
            
            ids=ids+data

ids_final=list(set(ids))
filename='HMMs_2nd_pass.txt'
with open(filename,'w') as f:
    for i in ids_final:
        f.write(i+'\n')

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  ./HMMs/GH10_80_no_outliers.hmm
# target sequence database:        all_non_GH_of_interest_1st_pass.fasta
# per-dom hits tabular output:     GH10_80_no_outliers_vs_all_non_GH_of_interest_1st_pass.txt
# show alignments in output:       no
# domain reporting threshold:      score >= 200
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       GH10_80_no_outliers  [M=487]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                  Description
    ------- ------ -----    ------- ------ -----   ---- --  

/tmp/ipykernel_114385/2450660144.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['IDs']=IDs


# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  ./HMMs/GH13_80_no_outliers.hmm
# target sequence database:        all_non_GH_of_interest_1st_pass.fasta
# per-dom hits tabular output:     GH13_80_no_outliers_vs_all_non_GH_of_interest_1st_pass.txt
# show alignments in output:       no
# domain reporting threshold:      score >= 91
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       GH13_80_no_outliers  [M=557]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                  Description
    ------- ------ -----    ------- ------ -----   ---- --  -

/tmp/ipykernel_114385/2450660144.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['IDs']=IDs
/tmp/ipykernel_114385/2450660144.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['IDs']=IDs
/tmp/ipykernel_114385/2450660144.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  ./HMMs/GH26_80_no_outliers.hmm
# target sequence database:        all_non_GH_of_interest_1st_pass.fasta
# per-dom hits tabular output:     GH26_80_no_outliers_vs_all_non_GH_of_interest_1st_pass.txt
# show alignments in output:       no
# domain reporting threshold:      score >= 90
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       GH26_80_no_outliers  [M=448]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                  Description
    ------- ------ -----    ------- ------ -----   ---- --  -

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  ./HMMs/GH39_80_no_outliers.hmm
# target sequence database:        all_non_GH_of_interest_1st_pass.fasta
# per-dom hits tabular output:     GH39_80_no_outliers_vs_all_non_GH_of_interest_1st_pass.txt
# show alignments in output:       no
# domain reporting threshold:      score >= 226
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       GH39_80_no_outliers  [M=526]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                   Description
    ------- ------ -----    ------- ------ -----   ---- -- 

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  ./HMMs/GH51_80_no_outliers.hmm
# target sequence database:        all_non_GH_of_interest_1st_pass.fasta
# per-dom hits tabular output:     GH51_80_no_outliers_vs_all_non_GH_of_interest_1st_pass.txt
# show alignments in output:       no
# domain reporting threshold:      score >= 342
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       GH51_80_no_outliers  [M=533]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                  Description
    ------- ------ -----    ------- ------ -----   ---- --  

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  ./HMMs/GH5_80_no_outliers.hmm
# target sequence database:        all_non_GH_of_interest_1st_pass.fasta
# per-dom hits tabular output:     GH5_80_no_outliers_vs_all_non_GH_of_interest_1st_pass.txt
# show alignments in output:       no
# domain reporting threshold:      score >= 55
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       GH5_80_no_outliers  [M=410]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence                      Description
    ------- ------ -----    ------- ------ -----   ---- --  

/tmp/ipykernel_114385/2450660144.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['IDs']=IDs
